In [0]:
import numpy as np
import pyspark as ps
import pandas as pd
import pyspark.pandas as psd
from pyspark.sql import SparkSession
from pyspark import SparkConf, SparkContext

spark = SparkSession.builder.getOrCreate()
spark.conf.set("spark.sql.ansi.enabled", "false")
spark.conf.set("spark.sql.execution.arrow.enabled", "true")

#Chargement
df_data = psd.read_csv("/Volumes/alpha_cloud_ai_workspace/default/exp-data/cf_asset_fi_brut.csv", sep=";")

df_sink_fund = pd.read_csv("/Volumes/alpha_cloud_ai_workspace/default/exp-data/sinking_fund.csv",
                            sep=";",
                            dtype={"Cusip": "string", "timing": np.int32, "prop": np.float32}
                        )


In [0]:
def add_cash_flow_columns(df, num_cashflows=1200):
    colones = dict()
    for i in range(num_cashflows):
        colones[f'cash_flow_{i+1}'] = 0
    return df.reindex(columns=list(df.columns) + list(colones.keys()), fill_value=0.0)

def add_pay_capital_columns(df, num_cashflows=1200):
    colones = dict()
    for i in range(num_cashflows):
        colones[f'pay_capital_{i+1}'] = 0
    return df.reindex(columns=list(df.columns) + list(colones.keys()), fill_value=0.0)

def compute_notional_cul_with_amortisation(maturity, notionnel, paycap):
    notional_cul = np.zeros(maturity + 1,dtype=float)
    notional_cul[0] = notionnel

    for i in range(maturity):
        notional_cul[i+1] = notional_cul[i] - paycap[k]
    return notional_cul

def compute_notional_cul(maturity,notionnel):
    notional_cul = np.zeros(maturity + 1,dtype=float)
    notional_cul[0] = notionnel

    for i in range(maturity):
        notional_cul[i+1] = notional_cul[i]
    return notional_cul

def compute_projection_pay_capital(serie):
    if serie["taux_flottant"] == 0: #On ne projete pas les titres à taux flottant
        if serie["cusip"] in df_sink_fund['Cusip'].values:
            amortization = df_sink_fund[df_sink_fund['Cusip'] == serie['cusip']]

            for i in range(serie["maturity"]):
                if i in amortization['timing'].values:
                    serie[f'pay_capital_{i+1}'] = amortization[amortization['timing'] == i]['prop'].values[0] * serie["notionnel"]
    return serie

In [0]:
def projection_cf_assert_opt(data,sink_fund):
    df_cash_flow = add_cash_flow_columns(data)
    df_pay_capital = add_pay_capital_columns(data)

    df_pay_capital = df_pay_capital.apply(compute_projection_pay_capital, axis=1)

    return df_pay_capital

result = projection_cf_assert_opt(df_data, df_sink_fund)
result.to_csv("/Volumes/alpha_cloud_ai_workspace/default/exp-data/test.csv", index=False)


In [0]:
  def projection_cf_asset(data, sink_fund):
   
    # Initialize the DataFrame
    cfs = np.zeros((len(data), 1200))
    pay_cap=np.zeros((len(data), 1200))
    # Iterate over each row in the data
    for j in range(len(data)):
        if data.iloc[j]['taux_flottant'] == 0: #On ne projete pas les titres à taux flottant
            tx_coupon = data.iloc[j]['coupon'] / data.iloc[j]['frequence']
            maturity = data.iloc[j]['maturity']
            # pay_cap = np.zeros(1200) #(maturity + 1) car on veut le cash flow de la dernière période
            notionnel_cul = np.zeros(maturity + 1)
            notionnel_cul[0] = data.iloc[j]['notionnel']
           
            for k in range(maturity):
                if data.iloc[j]['cusip'] in sink_fund['Cusip'].values: # Si le titre à une cédule d'amortissement
                    amortizer_cusip = sink_fund[sink_fund['Cusip'] == data.iloc[j]['cusip']]#
                    if k in amortizer_cusip['timing'].values:
                        pay_cap[j,k] = amortizer_cusip[amortizer_cusip['timing'] == k]['prop'].values[0] * data.iloc[j]['notionnel']
                    notionnel_cul[k + 1] = notionnel_cul[k] - pay_cap[j,k]
                    cfs[j, k] += pay_cap[j,k]
                else:
                    notionnel_cul[k + 1] = notionnel_cul[k]
           
            for i in range(data.iloc[j]['month_first_cf'], maturity + 1, int(12 / data.iloc[j]['frequence'])):
                cfs[j, i - 1] += tx_coupon * notionnel_cul[i - 1]
           
            cfs[j, maturity - 1] += notionnel_cul[maturity]
   
    # Convert the NumPy array to a DataFrame
    cfs_df = pd.DataFrame(cfs, columns=[f'Cash Flow {i}' for i in range(1, cfs.shape[1] + 1)])
    cfs_pay_cap_df = pd.DataFrame(pay_cap, columns=[f'Pay Capital {i}' for i in range(1, pay_cap.shape[1] + 1)])
    # Concatenate the data with the cash flows
    result = pd.concat([data.reset_index(drop=True), cfs_df], axis=1)
    results_cap = pd.concat([data.reset_index(drop=True), cfs_pay_cap_df], axis=1)
    return result,results_cap

In [0]:
result, result_cap= projection_cf_asset(df_asset, df_sinking)

#display(result)
display(result_cap)